In [1]:
# multi-head attention TFT

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Flatten, MultiHeadAttention
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
import gc

# 데이터셋 불러오기
file_path = 'C:\\Users\\dssal\\OneDrive\\바탕 화면\\이원병\\mp1.csv'
data = pd.read_csv(file_path)

# 필요 없는 컬럼 제거
data = data.drop(columns=['last_name, first_name'])

# 결측치 처리 (예: 0으로 대체)
data = data.fillna(0)

# 2020, 2021, 2022년에 모두 존재하는 player_id 추출
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2019 = set(data_2019['player_id'].unique())
player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2019 & player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2019, 2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# 필요한 컬럼 선택 (year 제외)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# 독립변수와 종속변수 분리
X = final[features].values
y = final[target].values

# 데이터 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# 시계열 데이터 형태로 변환
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # 시퀀스 길이 설정
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# 학습 데이터와 전체 데이터를 동일하게 설정
X_train, y_train = X_seq, y_seq

# 하이퍼파라미터 설정
state_sizes = [10, 20, 40, 80, 160, 240, 320]
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
minibatch_sizes = [32, 64, 128]
learning_rates = [0.0001, 0.001, 0.01]
max_gradient_norms = [0.01, 1, 100.0]
num_heads = [1, 2, 3, 4]

# 하이퍼파라미터 조합 생성
hyperparameter_combinations = list(product(state_sizes, dropout_rates, minibatch_sizes, learning_rates, max_gradient_norms, num_heads))

best_rmse = float('inf')
best_mae = float('inf')
best_mape = float('inf')
best_params = None

# TFT 모델 정의
class TFTModel(Model):
    def __init__(self, seq_length, feature_dim, num_heads, ff_dim, state_size, dropout_rate):
        super(TFTModel, self).__init__()
        self.multi_head_attention1 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm1 = LayerNormalization()
        self.multi_head_attention2 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm2 = LayerNormalization()
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(feature_dim)
        ])
        self.flatten = Flatten()
        self.dense1 = Dense(state_size, activation="relu")
        self.dropout1 = Dropout(dropout_rate)
        self.dense2 = Dense(state_size // 4, activation="relu")
        self.dense3 = Dense(1, activation="linear")

    def call(self, inputs):
        attn_output1 = self.multi_head_attention1(inputs, inputs)
        out1 = self.layer_norm1(inputs + attn_output1)
        attn_output2 = self.multi_head_attention2(out1, out1)
        out2 = self.layer_norm2(out1 + attn_output2)
        ffn_output = self.ffn(out2)
        flat_output = self.flatten(ffn_output)
        dense_output1 = self.dense1(flat_output)
        drop_output1 = self.dropout1(dense_output1)
        dense_output2 = self.dense2(drop_output1)
        return self.dense3(dense_output2)

# 하이퍼파라미터 튜닝
for state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads in hyperparameter_combinations:
    tf.keras.backend.clear_session()
    
    # 모델 인스턴스 생성
    model = TFTModel(seq_length=seq_length, feature_dim=X_train.shape[2], num_heads=num_heads, ff_dim=32, state_size=state_size, dropout_rate=dropout_rate)
    
    # 컴파일
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=max_gradient_norm)
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # 모델 학습
    history = model.fit(X_train, y_train, epochs=500, batch_size=minibatch_size, validation_split=0.2, verbose=0, callbacks=[early_stopping])
    
    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled = model.predict(X_2023_seq)

    # 스케일 복원
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))

    # MAE 계산
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    
    # MAPE 계산
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)

    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_mape = mape
        best_params = (state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads)

    print(f"Params: {state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")

    del model, history, y_pred_scaled, y_pred  # 필요한 변수를 명시적으로 삭제
    tf.keras.backend.clear_session()
    gc.collect()
    
print(f"Best RMSE: {best_rmse}, Best MAE: {best_mae}, Best MAPE: {best_mape}, Best Params: {best_params}")

2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 1), RMSE: 0.7916611280796013, MAE: 0.5976107848636688, MAPE: 0.1384839307542333
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 2), RMSE: 0.6982363587586861, MAE: 0.5028495440407404, MAPE: 0.11476270552137714
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 3), RMSE: 1.2979709056574056, MAE: 0.9890014610593283, MAPE: 0.21672782604125218
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 4), RMSE: 1.3517676586911702, MAE: 1.0561297869303872, MAPE: 0.2512342735814614
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 1), RMSE: 0.90359095580758, MAE: 0.6999628267590964, MAPE: 0.15794545895962483
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 2), RMSE: 0.6818588330598375, MAE: 0.5139680249350411, MAPE: 0.11584902078835334
2/2 [===

In [2]:
# basic TFT

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Flatten
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product

# 데이터셋 불러오기
file_path = 'C:\\Users\\co279\\mp1.csv'
data = pd.read_csv(file_path)

# 필요 없는 컬럼 제거
data = data.drop(columns=['last_name, first_name'])

# 결측치 처리 (예: 0으로 대체)
data = data.fillna(0)

# 2020, 2021, 2022년에 모두 존재하는 player_id 추출
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2019 = set(data_2019['player_id'].unique())
player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2019 & player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2019, 2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# 필요한 컬럼 선택 (year 제외)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# 독립변수와 종속변수 분리
X = final[features].values
y = final[target].values

# 데이터 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# 시계열 데이터 형태로 변환
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # 시퀀스 길이 설정
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# 학습 데이터와 전체 데이터를 동일하게 설정
X_train, y_train = X_seq, y_seq

# 하이퍼파라미터 설정
state_sizes = [10, 20, 40, 80, 160, 240, 320]
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
minibatch_sizes = [32, 64, 128]
learning_rates = [0.0001, 0.001, 0.01]
max_gradient_norms = [0.01, 1, 100.0]
num_heads = [1, 2, 3, 4]

# 하이퍼파라미터 조합 생성
hyperparameter_combinations = list(product(state_sizes, dropout_rates, minibatch_sizes, learning_rates, max_gradient_norms, num_heads))

best_rmse = float('inf')
best_mae = float('inf')
best_mape = float('inf')
best_params = None

# 모델 정의
class BasicTFTModel(Model):
    def __init__(self, seq_length, feature_dim, ff_dim, state_size, dropout_rate):
        super(BasicTFTModel, self).__init__()
        self.layer_norm1 = LayerNormalization()
        self.ffn1 = Dense(ff_dim, activation="relu")
        self.layer_norm2 = LayerNormalization()
        self.ffn2 = Dense(feature_dim)
        self.flatten = Flatten()
        self.dense1 = Dense(state_size, activation="relu")
        self.dropout1 = Dropout(dropout_rate)
        self.dense2 = Dense(state_size // 4, activation="relu")
        self.dense3 = Dense(1, activation="linear")

    def call(self, inputs):
        x = self.layer_norm1(inputs)
        x = self.ffn1(x)
        x = self.layer_norm2(x)
        x = self.ffn2(x)
        flat_output = self.flatten(x)
        dense_output1 = self.dense1(flat_output)
        drop_output1 = self.dropout1(dense_output1)
        dense_output2 = self.dense2(drop_output1)
        return self.dense3(dense_output2)

# 하이퍼파라미터 튜닝
for state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads in hyperparameter_combinations:
    tf.keras.backend.clear_session()
    
    # 모델 인스턴스 생성
    model = BasicTFTModel(seq_length=seq_length, feature_dim=X_train.shape[2], ff_dim=32, state_size=state_size, dropout_rate=dropout_rate)
    
    # 컴파일
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=max_gradient_norm)
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # 모델 학습
    history = model.fit(X_train, y_train, epochs=500, batch_size=minibatch_size, validation_split=0.2, verbose=0, callbacks=[early_stopping])
    
    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled = model.predict(X_2023_seq)

    # 스케일 복원
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))

    # MAE 계산
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    
    # MAPE 계산
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)

    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_mape = mape
        best_params = (state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads)

    print(f"Params: {state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")

print(f"Best RMSE: {best_rmse}, Best MAE: {best_mae}, Best MAPE: {best_mape}, Best Params: {best_params}")



2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 1), RMSE: 0.8009043544842864, MAE: 0.6203524043068055, MAPE: 0.13991971431157624
2/2 [==============================] - 0s 16ms/step
Params: (10, 0.1, 32, 0.0001, 0.01, 2), RMSE: 1.878366447924469, MAE: 1.4614217861114989, MAPE: 0.3461986253557367
2/2 [==============================] - 0s 17ms/step
Params: (10, 0.1, 32, 0.0001, 0.01, 3), RMSE: 1.2652809192130217, MAE: 0.9712729192158532, MAPE: 0.22830964103497295
2/2 [==============================] - 1s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 4), RMSE: 1.289385749965521, MAE: 1.0145962921021476, MAPE: 0.2410427011275192
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 1), RMSE: 2.4114730120260157, MAE: 2.104497058732169, MAPE: 0.4643624915787448
2/2 [==============================] - 0s 17ms/step
Params: (10, 0.1, 32, 0.0001, 1, 2), RMSE: 2.748824226191449, MAE: 2.290248346404423, MAPE: 0.5117925109843158
2/2 [==

In [2]:
# Full TFT

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, Flatten, MultiHeadAttention
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping
from itertools import product
import gc

# 데이터셋 불러오기
file_path = 'C:\\Users\\dssal\\OneDrive\\바탕 화면\\이원병\\mp1.csv'
data = pd.read_csv(file_path)

# 필요 없는 컬럼 제거
data = data.drop(columns=['last_name, first_name'])

# 결측치 처리 (예: 0으로 대체)
data = data.fillna(0)

# 2020, 2021, 2022년에 모두 존재하는 player_id 추출
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2019 = set(data_2019['player_id'].unique())
player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2019 & player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2019, 2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# 필요한 컬럼 선택 (year 제외)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# 독립변수와 종속변수 분리
X = final[features].values
y = final[target].values

# 데이터 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# 시계열 데이터 형태로 변환
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # 시퀀스 길이 설정
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# 학습 데이터와 전체 데이터를 동일하게 설정
X_train, y_train = X_seq, y_seq

# 하이퍼파라미터 설정
state_sizes = [10, 20, 40, 80, 160, 240, 320]
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
minibatch_sizes = [32, 64, 128]
learning_rates = [0.0001, 0.001, 0.01]
max_gradient_norms = [0.01, 1, 100.0]
num_heads = [1, 2, 3, 4]

# 하이퍼파라미터 조합 생성
hyperparameter_combinations = list(product(state_sizes, dropout_rates, minibatch_sizes, learning_rates, max_gradient_norms, num_heads))

best_rmse = float('inf')
best_mae = float('inf')
best_mape = float('inf')
best_params = None

# TFT 모델 정의 (정적 특성 제거)
class GatedResidualNetwork(tf.keras.layers.Layer):
    def __init__(self, input_dim, state_size, dropout_rate):
        super(GatedResidualNetwork, self).__init__()
        self.dense1 = Dense(state_size, activation="relu")
        self.dense2 = Dense(input_dim)  # 입력 차원에 맞춰 조정
        self.gate = Dense(input_dim, activation="sigmoid")
        self.layer_norm = LayerNormalization()
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dropout(x)
        x = self.dense2(x)
        gate_output = self.gate(inputs)
        gated_output = x * gate_output + inputs
        return self.layer_norm(gated_output)

# TFT 모델 정의 시 GatedResidualNetwork에 input_dim을 전달하도록 수정
class TFTModel(Model):
    def __init__(self, seq_length, feature_dim, num_heads, ff_dim, state_size, dropout_rate):
        super(TFTModel, self).__init__()

        # Temporal Attention Layers
        self.multi_head_attention1 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm1 = LayerNormalization()

        self.multi_head_attention2 = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dim)
        self.layer_norm2 = LayerNormalization()

        # Gated Residual Networks for gating mechanisms
        self.grn1 = GatedResidualNetwork(feature_dim, state_size, dropout_rate)
        self.grn2 = GatedResidualNetwork(feature_dim, state_size, dropout_rate)

        # Feed Forward Network
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(feature_dim)
        ])

        # Prediction Layers
        self.flatten = Flatten()
        self.dense1 = Dense(state_size, activation="relu")
        self.dropout1 = Dropout(dropout_rate)
        self.dense2 = Dense(state_size // 4, activation="relu")
        self.dense3 = Dense(1, activation="linear")

    def call(self, inputs):
        # Temporal Attention Layer 1 with Residual Connection and Gating
        attn_output1 = self.multi_head_attention1(inputs, inputs)
        out1 = self.layer_norm1(inputs + attn_output1)
        out1 = self.grn1(out1)

        # Temporal Attention Layer 2 with Residual Connection and Gating
        attn_output2 = self.multi_head_attention2(out1, out1)
        out2 = self.layer_norm2(out1 + attn_output2)
        out2 = self.grn2(out2)

        # Feed-Forward Network
        ffn_output = self.ffn(out2)

        # Flatten and predict
        flat_output = self.flatten(ffn_output)

        # Final Dense Layers
        dense_output1 = self.dense1(flat_output)
        drop_output1 = self.dropout1(dense_output1)
        dense_output2 = self.dense2(drop_output1)

        return self.dense3(dense_output2)

# 하이퍼파라미터 튜닝 및 학습
for state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads in hyperparameter_combinations:
    tf.keras.backend.clear_session()
    
    # 모델 인스턴스 생성
    model = TFTModel(seq_length=seq_length, feature_dim=X_train.shape[2], num_heads=num_heads, ff_dim=32, state_size=state_size, dropout_rate=dropout_rate)
    
    # 컴파일
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=max_gradient_norm)
    model.compile(
        loss="mse",
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # 모델 학습
    history = model.fit(X_train, y_train, epochs=500, batch_size=minibatch_size, validation_split=0.2, verbose=0, callbacks=[early_stopping])
    
    # 2023년 데이터 필터링
    data_23 = common_data[common_data['year'] == 2023]

    # 2023년 데이터 스케일링
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled = model.predict(X_2023_seq)

    # 스케일 복원
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_23[target].values

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_test_actual[:len(y_pred)], y_pred))

    # MAE 계산
    mae = mean_absolute_error(y_test_actual[:len(y_pred)], y_pred)
    
    # MAPE 계산
    mape = mean_absolute_percentage_error(y_test_actual[:len(y_pred)], y_pred)

    if rmse < best_rmse:
        best_rmse = rmse
        best_mae = mae
        best_mape = mape
        best_params = (state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads)

    print(f"Params: {state_size, dropout_rate, minibatch_size, learning_rate, max_gradient_norm, num_heads}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")

    del model, history, y_pred_scaled, y_pred  # 필요한 변수를 명시적으로 삭제
    tf.keras.backend.clear_session()
    gc.collect()
    
print(f"Best RMSE: {best_rmse}, Best MAE: {best_mae}, Best MAPE: {best_mape}, Best Params: {best_params}")

2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 1), RMSE: 0.8368456952731133, MAE: 0.6432021026005821, MAPE: 0.1576372867328787
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 2), RMSE: 0.7199624686631475, MAE: 0.5655657999856132, MAPE: 0.13214506975083276
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 3), RMSE: 1.0257336074134027, MAE: 0.7788730186129375, MAPE: 0.17647956650815055
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 0.01, 4), RMSE: 0.8392908570141149, MAE: 0.6681612848857091, MAPE: 0.1500733617928006
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 1), RMSE: 1.3831766296054686, MAE: 1.0415702874319894, MAPE: 0.22078178865927958
2/2 [==============================] - 0s 0s/step
Params: (10, 0.1, 32, 0.0001, 1, 2), RMSE: 1.2302449252811256, MAE: 0.9218601464468337, MAPE: 0.2064680831750622
2/2 [==